In [189]:
import hanlp

In [190]:
# 所有pre-trained tokenizers
hanlp.pretrained.tok.ALL

{'SIGHAN2005_PKU_CONVSEG': 'https://file.hankcs.com/hanlp/tok/sighan2005-pku-convseg_20200110_153722.zip',
 'SIGHAN2005_MSR_CONVSEG': 'https://file.hankcs.com/hanlp/tok/convseg-msr-nocrf-noembed_20200110_153524.zip',
 'CTB6_CONVSEG': 'https://file.hankcs.com/hanlp/tok/ctb6_convseg_nowe_nocrf_20200110_004046.zip',
 'PKU_NAME_MERGED_SIX_MONTHS_CONVSEG': 'https://file.hankcs.com/hanlp/tok/pku98_6m_conv_ngram_20200110_134736.zip',
 'LARGE_ALBERT_BASE': 'https://file.hankcs.com/hanlp/tok/large_corpus_cws_albert_base_20211228_160926.zip',
 'SIGHAN2005_PKU_BERT_BASE_ZH': 'https://file.hankcs.com/hanlp/tok/sighan2005_pku_bert_base_zh_20201231_141130.zip',
 'COARSE_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/tok/coarse_electra_small_20220616_012050.zip',
 'FINE_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/tok/fine_electra_small_20220615_231803.zip',
 'CTB9_TOK_ELECTRA_SMALL': 'https://file.hankcs.com/hanlp/tok/ctb9_electra_small_20220215_205427.zip',
 'CTB9_TOK_ELECTRA_BASE': 'http:/

In [191]:
# 加载模型
tok = hanlp.load("https://file.hankcs.com/hanlp/tok/coarse_electra_small_20220220_013548.zip")
tok

c:\Users\duanm\anaconda3\envs\nlp_hanlp\lib\site-packages\hanlp\common\torch_component.py:100: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model_.load_state_dict(torc

In [192]:
# 执行分词
tok(['商品和服务。', '晓美焰来到北京立方庭参观自然语义科技公司'])  

[['商品', '和', '服务', '。'], ['晓美焰', '来到', '北京立方庭', '参观', '自然语义科技公司']]

### 自定义词典

In [193]:
tok.dict_combine, tok.dict_force  # 默认tok.dict_combine和tok.dict_force均为None

(None, None)

In [194]:
tok("首相和川普通电话")

['首相', '和', '川', '普通', '电话']

In [195]:
# 强制模式
tok.dict_force = {'川普'}
tok(
    ["首相和川普通电话",  # 川普正常被分词
     "银川普通人与川普通电话讲四川普通话"  # 匹配不该分出来的自定义词语(导致歧义)
     ]
)

[['首相', '和', '川普', '通', '电话'],
 ['银', '川普', '通人', '与', '川普', '通', '电话', '讲', '四', '川普', '通话']]

In [196]:
# 强制校正模式(对每种可能产生歧义的语境,截取一个片段执行校正)
tok.dict_force = {'川普通电话': ['川普', '通', '电话']}  
tok(["首相和川普通电话", "银川普通人与川普通电话讲四川普通话"])

[['首相', '和', '川普', '通', '电话'],
 ['银川', '普通人', '与', '川普', '通', '电话', '讲', '四川', '普通话']]

In [197]:
tok.dict_force = tok.dict_combine = None
# 提供给神经网络足够的上下文线索,告诉神经网络"川普是美国总统"
print(tok([
    # 虽然词典对"川普"没有施加任何影响,但是更丰富的上下文促进了神经网络对语境的理解,使其得出了正确的结果
    "首相和川普通电话，川普是美国总统。",
    "银川普通人与川普通电话讲四川普通话，川普是美国总统。"
]
))

[['首相', '和', '川普', '通', '电话', '，', '川普', '是', '美国', '总统', '。'], ['银川', '普通人', '与', '川普', '通', '电话', '讲', '四川', '普通话', '，', '川普', '是', '美国', '总统', '。']]


In [198]:
tok.dict_force = None
tok.dict_combine = None
# 合并模式(dict_combine会在统计模型的分词结果上执行最长匹配并合并匹配到的词条)
tok.dict_combine = {'美国总统'}
tok([
    # 初始分词结果为[..., "美国", "总统", ...] ---合并后为---> [..., "美国总统", ...]
    "首相和川普通电话，川普是美国总统。",
    # 初始分词结果为["美国", "总统筹部", ...] ---不会被合共---> ["美国", "总统筹部", ...] 
    "美国总统筹部部长是谁？"
]
)

[['首相', '和', '川普', '通', '电话', '，', '川普', '是', '美国总统', '。'],
 ['美国', '总统筹部', '部长', '是', '谁', '？']]

In [199]:
tok.dict_combine = {
    # 有空格、制表符等(Transformer tokenizer去掉的字符)的词语需要用tuple的形式提供
    ('iPad', 'Pro'),
    '2个空格'
}

# 初始分词结果为[..., 'iPad', 'Pro', ..., 'iPad', 'Pro',...] ---合并后为---> [..., 'iPad  Pro', ..., 'iPad  Pro',...] 
tok("如何评价iPad Pro ？iPad  Pro有2个空格")

['如何', '评价', 'iPad Pro', '？', 'iPad  Pro', '有', '2个空格']

In [200]:
tok.config.output_spans = True  # 输出单词位置
sent = '2021 年\nHanLPv2.1 为生产环境带来次世代最先进的多语种NLP技术。'
word_offsets = tok(sent)
print(word_offsets)
tok.config.output_spans = False

[['2021', 0, 4], ['年', 5, 6], ['HanLPv2.1', 7, 16], ['为', 17, 18], ['生产', 18, 20], ['环境', 20, 22], ['带来', 22, 24], ['次', 24, 25], ['世代', 25, 27], ['最', 27, 28], ['先进', 28, 30], ['的', 30, 31], ['多', 31, 32], ['语种', 32, 34], ['NLP', 34, 37], ['技术', 37, 39], ['。', 39, 40]]
